In [12]:
# from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense, CuDNNGRU, LSTM, GRU, Dropout, Concatenate, Input, Flatten
from keras.layers import *
from keras.initializers import Constant
from keras import Sequential
import keras.backend as K
import numpy as np
from keras.backend import epsilon
import ipdb
import random
from keras_layer_normalization import LayerNormalization
from keras import regularizers
from keras.models import Model


EMBEDDING_DIM = 300
INPUT_LENGTH = 1
MELODY_VEC_LENGTH = 150

class ModelFoo:
    def __init__(self, tokenizer, embedding_matrix,
                 rnn_units=50,
                 bidirectional=True,
                 rnn_type='lstm',
                 dropout=0.3,
                 show_summary=True):
        rnn_types = {
            'lstm': CuDNNLSTM,
            'gru': CuDNNGRU
        }
        rnn_type = rnn_types[rnn_type]

        # load pre-trained word embeddings into an Embedding layer
        # note that we set trainable = False so as to keep the embeddings fixed
        num_words = len(tokenizer.word_index) + 1
        embedding_layer = Embedding(num_words,
                                    EMBEDDING_DIM,
                                    embeddings_initializer=Constant(embedding_matrix),
                                    input_length=INPUT_LENGTH,
                                    trainable=True)
        lyrics_input = Input(shape=(INPUT_LENGTH,))
        melody_input = Input(shape=(MELODY_VEC_LENGTH,))
        
#         melody = Flatten()(melody_input)
        lyrics = embedding_layer(lyrics_input)
        lyrics = Flatten()(lyrics)
        lyrics = Dropout(dropout)(lyrics)
        combined = Concatenate()([lyrics, melody_input])
        combined = Reshape((1, EMBEDDING_DIM + MELODY_VEC_LENGTH))(combined)
        combined = rnn_type(rnn_units)(combined)
        if bidirectional:
            combined = Bidirectional(combined)
#         combined = LayerNormalization()(combined)
        combined = Dense(num_words, kernel_regularizer=regularizers.l2(0.1), activation='softmax')(combined)
        model = Model(inputs=[lyrics_input, melody_input], outputs=[combined])

        if show_summary:
            model.summary()

        self.model = model
        self.tokenizer = tokenizer

        
    def train(self, X, y, epochs=5, batch_size=32, callbacks=[]):
        model = self.model
        # compile network
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[_perplexity])
        # fit network
        model.fit(X, y, 
                  epochs=epochs,
                  batch_size=batch_size,
                  verbose=1, 
                  shuffle=True,
                  validation_split=0.1,
                  callbacks=callbacks)

        
    def predict(self, first_word, n_words):
        in_text, result = first_word, first_word
        # generate a fixed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = self.tokenizer.texts_to_sequences([in_text])[0]
            encoded = np.array(encoded)
            
            words_probs = self.model.predict_proba(encoded, verbose=0)[0]
            
            # get 2 arrays of probs and word_tokens
            words_probs_enu = list(enumerate(words_probs))
            words_probs_sorted = sorted(words_probs_enu, key=lambda x: x[1], reverse=True) # sorting in descending order
            words_tokens, words_probs = list(zip(*words_probs_sorted))
            # normalizre to sum 1 
            words_probs = np.array(words_probs, dtype=np.float64)
            words_probs /= words_probs.sum().astype(np.float64)
            word_token = np.random.choice(words_tokens, p=words_probs)
            
            # map predicted word index to word
            out_word = get_word(word_token, self.tokenizer)
            # append to input
            in_text, result = out_word, result + ' ' + out_word
        return result      

        
    def predict_beam(self, first_word, n_words, B=3):
        tokenizer = self.tokenizer
        model = self.model
        in_text, result = first_word, [first_word]
        encoded = get_encoded(in_text, tokenizer)
        beam_sequences_scores = [[encoded, 0]]

        while len(result) < n_words:
            all_candidates = []
            beam_sequences_scores = self.beam_step(beam_sequences_scores, B)
            for seq_score in beam_sequences_scores:
                seq_scores = self.beam_step([seq_score], B)
                all_candidates.append(seq_scores)
            flatten = lambda lst: [item for sublist in lst for item in sublist]
            beam_sequences_scores = sorted(flatten(all_candidates), reverse=True, key=lambda tup: tup[1])[:B]
            result, _= beam_sequences_scores[0]
        
        words = [get_word(token, self.tokenizer) for token in result]
        return ' '.join(words)
    
    
    def beam_step(self, beam_sequences_scores, B):            
        all_candidates = []
        for seq, score in beam_sequences_scores: # for each sequence
            # predict top B words
            seq_pad = pad_sequences([[sample] for sample in seq], maxlen=INPUT_LENGTH)
            words_probs = self.model.predict_proba(seq_pad, verbose=0)[0]
            words_probs_enu = list(enumerate(words_probs))
            words_probs_sorted = sorted(words_probs_enu, key=lambda x: x[1], reverse=True) # sorting in descending order
            top_b_words_probs = words_probs_sorted[:B] # top B words with max probability
            # for each prob in top B words, create a candidate
            for token, prob in top_b_words_probs: 
                word_token = token
                candidate = [np.append(seq, word_token), score + np.log(prob + epsilon())] # todo: word_token 
                all_candidates.append(candidate)
        # take candidates with max score
        beam_sequences_scores = sorted(all_candidates, reverse=True, key=lambda tup: tup[1])[:B]
        return beam_sequences_scores

    
def get_encoded(text, tokenizer):
    encoded = tokenizer.texts_to_sequences([text])[0]
    encoded = np.array(encoded)
    return encoded

def get_word(index, tokenizer):
    for word, idx in tokenizer.word_index.items():
         if idx == index:
            return word

def _perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.pow(2.0, cross_entropy)
    return perplexity


In [9]:
import src.dataset as ds
from src.embeddings import extract_embedding_weights

In [10]:
x, y, tokenizer, songs = joblib.load("x_y_tokenizer_songs.jblib")
embedding_matrix = extract_embedding_weights()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 614/614 [00:01<00:00, 362.82it/s]


In [24]:
model1 = ModelFoo(tokenizer, embedding_matrix, rnn_type='gru', rnn_units=50, bidirectional=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 300)       2247300     input_5[0][0]                    
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 300)          0           embedding_3[0][0]                
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 300)          0           flatten_3[0][0]                  
__________________________________________________________________________________________________
input_6 (I

In [26]:
model1.train([x,songs], y, epochs=10)

Train on 171249 samples, validate on 19028 samples
Epoch 1/10
171249/171249 [==============================] - 28s 163us/step - loss: 6.6572 - _perplexity: 195.9328 - val_loss: 6.4176 - val__perplexity: 316.2075
Epoch 2/10
171249/171249 [==============================] - 26s 151us/step - loss: 5.8648 - _perplexity: 167.8010 - val_loss: 6.2426 - val__perplexity: 939.6688
Epoch 3/10
171249/171249 [==============================] - 26s 151us/step - loss: 5.6493 - _perplexity: 192.1527 - val_loss: 6.2647 - val__perplexity: 3407.3534
Epoch 4/10
171249/171249 [==============================] - 26s 151us/step - loss: 5.5690 - _perplexity: 218.1963 - val_loss: 6.2647 - val__perplexity: 4207.7273
Epoch 5/10
171249/171249 [==============================] - 26s 153us/step - loss: 5.5361 - _perplexity: 239.3797 - val_loss: 6.2525 - val__perplexity: 4233.2446
Epoch 6/10
171249/171249 [==============================] - 26s 153us/step - loss: 5.5208 - _perplexity: 256.2517 - val_loss: 6.2404 - val__p

In [28]:
?model1.predict

Signature: model1.predict(first_word, n_words)
Docstring: <no docstring>
File:      c:\users\eli\workspace\deep-learning-intro\assignment3\<ipython-input-12-227bc8e531a9>
Type:      method


In [29]:
model1.predict(first_word='deep', n_words=10)

AttributeError: 'Model' object has no attribute 'predict_proba'

In [34]:
songs[0].shape

(150,)

In [40]:
model1.model.predict([[5], [songs[0]]])

array([[1.3026547e-09, 1.2094216e-06, 7.8628642e-07, ..., 9.0679328e-07,
        1.2554902e-06, 1.1776601e-03]], dtype=float32)